#Text generation with GPT-2

###Imports

In [1]:
!pip install pandas
!pip install keras
!pip install tensorflow
!pip install nltk
!pip install torch
!pip3 install -U scikit-learn scipy matplotlib
!pip install transformers
!pip install datasets
!pip install --user -U nltk


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip insta

In [2]:
import csv
import pandas as pd
import numpy as np

import string
import re

from keras.preprocessing.text import Tokenizer
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk import ngrams
from nltk.tokenize import word_tokenize


import torch, os, re, pandas as pd, json
from sklearn.model_selection import train_test_split
from transformers import DataCollatorForLanguageModeling, DataCollatorWithPadding, GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, AutoConfig
from datasets import Dataset

2023-05-06 20:14:16.495606: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/marinakireeva/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/marinakireeva/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/marinakireeva/nltk_data...


True

##Data preprocessing

###Import data from csv

In [3]:
tweets=pd.read_csv("covid19_tweets.csv",delimiter=",")

In [4]:
tweets=tweets[["text"]]

In [5]:
tweets.head(5)

,text
0,If I smelled the scent of hand sanitizers toda...
1,Hey @Yankees @YankeesPR and @MLB - wouldn't it...
2,@diane3443 @wdunlap @realDonaldTrump Trump nev...
3,@brookbanktv The one gift #COVID19 has give me...
4,25 July : Media Bulletin on Novel #CoronaVirus...


###To lower case

In [6]:
lowered = tweets['text'].str.lower()
tweets['lowered'] = lowered

In [7]:
tweets.head(5)

,text,lowered
0,If I smelled the scent of hand sanitizers toda...,if i smelled the scent of hand sanitizers toda...
1,Hey @Yankees @YankeesPR and @MLB - wouldn't it...,hey @yankees @yankeespr and @mlb - wouldn't it...
2,@diane3443 @wdunlap @realDonaldTrump Trump nev...,@diane3443 @wdunlap @realdonaldtrump trump nev...
3,@brookbanktv The one gift #COVID19 has give me...,@brookbanktv the one gift #covid19 has give me...
4,25 July : Media Bulletin on Novel #CoronaVirus...,25 july : media bulletin on novel #coronavirus...


###Clean text punctuation marks

In [8]:
tweets['lowered']=[re.sub(r'[^\w\s]','',text) for text in tweets['lowered']]

In [9]:
tweets.head(5)

,text,lowered
0,If I smelled the scent of hand sanitizers toda...,if i smelled the scent of hand sanitizers toda...
1,Hey @Yankees @YankeesPR and @MLB - wouldn't it...,hey yankees yankeespr and mlb wouldnt it have...
2,@diane3443 @wdunlap @realDonaldTrump Trump nev...,diane3443 wdunlap realdonaldtrump trump never ...
3,@brookbanktv The one gift #COVID19 has give me...,brookbanktv the one gift covid19 has give me i...
4,25 July : Media Bulletin on Novel #CoronaVirus...,25 july media bulletin on novel coronavirusup...


###Tokenization
Breaking down text into smaller pieces

In [10]:
tokened = tweets.apply(lambda row: nltk.word_tokenize(row['lowered']), axis=1)
tweets['tokenized'] = tokened

In [11]:
tweets.head(5)

,text,lowered,tokenized
0,If I smelled the scent of hand sanitizers toda...,if i smelled the scent of hand sanitizers toda...,"[if, i, smelled, the, scent, of, hand, sanitiz..."
1,Hey @Yankees @YankeesPR and @MLB - wouldn't it...,hey yankees yankeespr and mlb wouldnt it have...,"[hey, yankees, yankeespr, and, mlb, wouldnt, i..."
2,@diane3443 @wdunlap @realDonaldTrump Trump nev...,diane3443 wdunlap realdonaldtrump trump never ...,"[diane3443, wdunlap, realdonaldtrump, trump, n..."
3,@brookbanktv The one gift #COVID19 has give me...,brookbanktv the one gift covid19 has give me i...,"[brookbanktv, the, one, gift, covid19, has, gi..."
4,25 July : Media Bulletin on Novel #CoronaVirus...,25 july media bulletin on novel coronavirusup...,"[25, july, media, bulletin, on, novel, coronav..."


###Cleaning text from stop words

In [12]:
noise = stopwords.words('english')
withoutstop = tweets['tokenized'].apply(lambda x: [item for item in x if item not in noise])
without_stop = []
for a in withoutstop:    
    without_stop.append(", ".join(a))
tweets['without_stop'] = without_stop

In [13]:
tweets.head(5)

,text,lowered,tokenized,without_stop
0,If I smelled the scent of hand sanitizers toda...,if i smelled the scent of hand sanitizers toda...,"[if, i, smelled, the, scent, of, hand, sanitiz...","smelled, scent, hand, sanitizers, today, someo..."
1,Hey @Yankees @YankeesPR and @MLB - wouldn't it...,hey yankees yankeespr and mlb wouldnt it have...,"[hey, yankees, yankeespr, and, mlb, wouldnt, i...","hey, yankees, yankeespr, mlb, wouldnt, made, s..."
2,@diane3443 @wdunlap @realDonaldTrump Trump nev...,diane3443 wdunlap realdonaldtrump trump never ...,"[diane3443, wdunlap, realdonaldtrump, trump, n...","diane3443, wdunlap, realdonaldtrump, trump, ne..."
3,@brookbanktv The one gift #COVID19 has give me...,brookbanktv the one gift covid19 has give me i...,"[brookbanktv, the, one, gift, covid19, has, gi...","brookbanktv, one, gift, covid19, give, appreci..."
4,25 July : Media Bulletin on Novel #CoronaVirus...,25 july media bulletin on novel coronavirusup...,"[25, july, media, bulletin, on, novel, coronav...","25, july, media, bulletin, novel, coronavirusu..."


###Lemmatization 
Reducing words to its base

In [14]:
lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    tokens = nltk.word_tokenize(text)
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return ' '.join(lemmatized_tokens)

In [15]:
tweets['lemmatized'] = tweets['without_stop'].apply(lemmatize_text)

In [16]:
tweets['text']=tweets['lemmatized']
tweets=tweets[['text']]

In [17]:
tweets.head(5)

,text
0,"smelled , scent , hand , sanitizers , today , ..."
1,"hey , yankee , yankeespr , mlb , wouldnt , mad..."
2,"diane3443 , wdunlap , realdonaldtrump , trump ..."
3,"brookbanktv , one , gift , covid19 , give , ap..."
4,"25 , july , medium , bulletin , novel , corona..."


###Removing commas

In [18]:
tweets['text']=[re.sub(r'[^\w\s]','',text) for text in tweets['text']]

In [19]:
tweets.head(5)

,text
0,smelled scent hand sanitizers today someo...
1,hey yankee yankeespr mlb wouldnt made se...
2,diane3443 wdunlap realdonaldtrump trump ne...
3,brookbanktv one gift covid19 give appreci...
4,25 july medium bulletin novel coronavirus...


##Splitiing data set on test and validation dataframes

In [156]:
df_train, df_val = train_test_split(tweets, train_size = 0.03, random_state = 77)

In [157]:
len(df_train)

5373

In [158]:
len(df_val)

173735

##Adding special tokens to the dataset

In [159]:
bos = '<|endoftext|>'
eos = '<|EOS|>'
pad = '<|pad|>'

df_train ['text'] = bos + ' ' + df_train['text'] + ' ' + eos
df_val['text'] = bos + ' ' + df_val['text'] + ' ' + eos

In [160]:
train = Dataset.from_pandas(df_train[['text']])
val = Dataset.from_pandas(df_val[['text']])

###Importing tokenizer

In [161]:
base_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
base_tokenizer.add_special_tokens({'pad_token': '[PAD]'})
base_tokenizer.pad_token = '[PAD]'

In [162]:
def tokenize_function(examples):
    return base_tokenizer(
        examples['text'],
        padding='max_length',   
        truncation=True,        
        max_length=10          
    )


tokenized_train_dataset = train.map(
    tokenize_function,
    batched=True,
    num_proc=5,
    remove_columns=['text'],
)
tokenized_val_dataset = val.map(
    tokenize_function,
    batched=True,
    num_proc=5,
    remove_columns=['text'],
)

Map (num_proc=5):   0%|          | 0/5373 [00:00<?, ? examples/s]

Map (num_proc=5):   0%|          | 0/173735 [00:00<?, ? examples/s]

In [163]:
# Example of the result of the tokenization process with padding
base_tokenizer.decode(tokenized_train_dataset['input_ids'][0])

'<|endoftext|> 13  year  working  paramedic '

##Model training

In [164]:
model = GPT2LMHeadModel.from_pretrained('gpt2')

In [165]:
model_headlines_path = './model_headlines_news' #заменить
 
training_args = TrainingArguments(
    output_dir=model_headlines_path,          
    num_train_epochs=6,              
    per_device_train_batch_size=16,  
    per_device_eval_batch_size=16,   
    warmup_steps=200,                
    weight_decay=0.01,               
    logging_dir=model_headlines_path,           
    prediction_loss_only=True,
    save_steps=10000 
)


In [166]:
data_collator = DataCollatorForLanguageModeling(
        tokenizer=base_tokenizer,
        mlm=False
    )


In [167]:
model.config.max_length

20

In [169]:
trainer = Trainer(
    model=model,                         
    args=training_args,                  
    data_collator=data_collator,
    train_dataset=tokenized_train_dataset,        
    eval_dataset=tokenized_val_dataset            
)
trainer.train()


Step,Training Loss
500,4.114400
1000,3.613000
1500,3.225800
2000,3.020500


TrainOutput(global_step=2016, training_loss=3.48923126856486, metrics={'train_runtime': 3596.7902, 'train_samples_per_second': 8.963, 'train_steps_per_second': 0.56, 'total_flos': 164522119680000.0, 'train_loss': 3.48923126856486, 'epoch': 6.0})

In [170]:
trainer.save_model()
base_tokenizer.save_pretrained(model_headlines_path)

('./model_headlines_news/tokenizer_config.json',
 './model_headlines_news/special_tokens_map.json',
 './model_headlines_news/vocab.json',
 './model_headlines_news/merges.txt',
 './model_headlines_news/added_tokens.json')

In [171]:
trainer.evaluate()

KeyboardInterrupt: 

##Generating news with pretrained model

In [173]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load the fine-tuned model
model = GPT2LMHeadModel.from_pretrained('./model_headlines_news')

# Load the tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Set the model to evaluation mode
model.eval()

# Generate texts
prompt = "covid"
input_ids = tokenizer.encode(prompt, return_tensors='pt')
output = model.generate(input_ids, max_length=50, num_return_sequences=1)

# Decode the generated output
decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)

# Print the generated output
print(decoded_output)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


covid19  pandemic  accelerated  covid19  pandemic  accelerated  pandemic  accelerated  pandemic  accelerated  pandemic  accelerated  pandemic  accelerated  pandemic  accelerated  pandemic  accelerated  pand


In [179]:
prompt = "covid19"
input_ids = tokenizer.encode(prompt, return_tensors='pt')
output = model.generate(input_ids, max_length=20, num_return_sequences=1)

# Decode the generated output
decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)

# Print the generated output
print(decoded_output)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


covid19  pandemic  accelerated  covid19  pandemic  accelerated  pand
